In [1]:
# 데이터 다운로드 링크 : http://www.manythings.org/anki
# Base Code : https://wikidocs.net/24996
import pandas as pd
lines = pd.read_csv('dataset/fra.txt', names=['src', 'tar','drop'], sep='\t')
lines.drop(['drop'], inplace=True, axis=1)
len(lines)


170651

In [2]:
lines = lines[0:60000]
lines.sample(10)

,src,tar
34875,I want some of those.,Je veux quelques-uns de ceux-ci.
39046,Do you have any money?,As-tu un quelconque argent ?
35177,I'm all out of ideas.,Je suis dépourvu d'idées.
42012,Let's call it a night.,On va s'arrêter là pour ce soir.
53234,I just got your message.,Je viens d'avoir ton message.
6683,We will fight.,Nous combattrons.
40948,I think they're lying.,Je pense qu'ils mentent.
28909,I need to know more.,J'ai besoin d'en savoir plus.
49105,The meeting was closed.,La réunion fut clôturée.
17853,Get into your car.,File dans ta voiture !


In [3]:
lines['tar'] = lines['tar'].apply(lambda x : '\t ' + x + ' \n')
lines.sample(5)
# 시작을 의미하는 심볼과 종료를 의미하는 심볼을 각각 \t 와 \n으로 표현.

,src,tar
4120,Let's go now.,\t Allons-y maintenant. \n
46584,I fully agree with you.,\t Je suis complètement d'accord avec toi. \n
6170,Quit gambling.,\t Arrête de jouer. \n
33446,He has nothing to do.,\t Il n'a rien à faire. \n
33549,He made up an excuse.,\t Il a inventé une excuse. \n


In [4]:
# 글자 집합 구축
# src_vocab = set()
# for line in lines['src']:
#     for char in line:
#         src_vocab.add(char)

# tar_vocab=set()
# for line in lines['tar']:
#     for char in line:
#         tar_vocab.add(char)

src_vocab = set([a for b in lines['src'] for a in b]) # 위 코드와 결과는 같음
tar_vocab = set([a for b in lines['tar'] for a in b])

# 단어 집합이 아니라 글자 집합이라고 하는 이유는 토큰 단위가 단어가 아니라 글자이기 때문


In [5]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

79
106


In [6]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

# 글자 집합에 글자 단위로 저장 된 것을 확인할 수 있다.

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w']


In [7]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)
# 정수 인코딩을 위한 과정


{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, '’': 77, '€': 78}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 3

In [8]:
encoder_input = []
for line in lines['src']:
    temp_X = []
    for W in line:
        temp_X.append(src_to_index[W])
    encoder_input.append(temp_X)
print('영어 정수 인코딩 : ',encoder_input[:5])


decoder_input = []
for line in lines['tar']:
    temp_X = []
    for W in line:
        temp_X.append(tar_to_index[W])
    decoder_input.append(temp_X)
print('프랑스어 정수 인코딩 : ', decoder_input[:5])
# 정수 인코딩 결과 5개 샘플
# 프랑스어는 <sos> 때문에 모두 앞에 1이 붙음.


영어 정수 인코딩 :  [[30, 64, 10], [31, 58, 10], [31, 58, 10], [41, 70, 63, 2], [41, 70, 63, 2]]
프랑스어 정수 인코딩 :  [[1, 3, 48, 53, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 14, 3, 2], [1, 3, 29, 67, 73, 70, 71, 105, 4, 3, 2], [1, 3, 29, 67, 73, 70, 57, 78, 105, 4, 3, 2]]


In [9]:
# decoder_target = []
# for line in lines.tar:
#     t=0
#     temp_X = []
#     for w in line:
#       if t>0:
#         temp_X.append(tar_to_index[w])
#       t=t+1
#     decoder_target.append(temp_X)
# print(decoder_target[:5])

decoder_target = [k[1:] for k in decoder_input] # 위와 같은 코드
decoder_target[:5]

# 실제값에는 시작 심볼에 해당하는 <sos>가 필요 없으므로 1을 지워준다.

[[3, 48, 53, 3, 4, 3, 2],
 [3, 45, 53, 64, 73, 72, 3, 4, 3, 2],
 [3, 45, 53, 64, 73, 72, 14, 3, 2],
 [3, 29, 67, 73, 70, 71, 105, 4, 3, 2],
 [3, 29, 67, 73, 70, 57, 78, 105, 4, 3, 2]]

In [10]:
max_src_len = max([len(line) for line in lines['src']])
max_tar_len = max([len(line) for line in lines['tar']])
print(max_src_len)
print(max_tar_len)

# 패딩 작업을 위해 가장 긴 길이를 가진 문장의 길이를 구함.

25
76


In [11]:
from keras.preprocessing.sequence import pad_sequences
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')
# 영어와 프랑스의 길이는 하나의 쌍이라고 하더라도 전부 다르므로 패딩 할 때도 동일하게 맞춰줄 필요가 없다.
# 영어 데이터는 영어 샘플끼리 프랑스어는 프랑스어 끼리 맞추어 패딩

Using TensorFlow backend.
C:\Users\JeongMyeong\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\JeongMyeong\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\JeongMyeong\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\JeongMyeong\Anaconda3\lib\site-packages\tensorflow\py

In [12]:
from keras.utils import to_categorical
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

# 글자단위 번역기므로 워드 임베딩은 별도로 사용 안함.
# 단어 단위로 바꾸어 워드임베딩을 활용한 seq2seq도 시도예정

array([[[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.